# Fábrica de Snapshots (Snapshot Factory)

Este notebook foi desenhado para rodar no **Google Colab** (com GPU).
Ele cria a estrutura de pastas recomendada e treina modelos Generalistas e Especialistas em lote.

> **Dica Colab**: Descomente a linha de instalação apropriada (Pública ou Privada) abaixo.

In [ ]:
# --- INSTALAÇÃO ---
# Descomente a linha abaixo para instalar no Colab/Kaggle
# !pip install --force-reinstall git+https://github.com/aretw0/predicoes-loterias-caixa.git


In [ ]:
import os
import sys
import pathlib
import pandas as pd

# --- LÓGICA DE IMPORTAÇÃO ---
from loterias.megasena import MegaSena
from loterias.models import TransformerModel, CatBoostModel, LSTMModel
print("✅ Módulos importados com sucesso.")

In [ ]:
# 2. Criar Estrutura de Pastas
games = ['megasena', 'lotofacil', 'quina']
types = ['generalistas', 'especialistas/virada', 'especialistas/acumulados']

for game in games:
    for t in types:
        path = f"snapshots/{game}/{t}"
        os.makedirs(path, exist_ok=True)
        print(f"Criado: {path}")

In [ ]:
# 3. Carregar Dados Reais
lottery = MegaSena()
df_full = lottery.preprocess_data()
print(f"Total Concursos: {len(df_full)}")

## 4. Treinar Generalistas (Todos os dados)
Use parâmetros agressivos aqui.

In [ ]:
import tensorflow as tf
print("--- Treinando Transformer Generalista ---")
model_gen = TransformerModel(1, 60, 6)
# High-Compute Params
history = model_gen.train(df_full, epochs=100, batch_size=64, 
                          head_size=256, num_heads=4, ff_dim=256)

save_path = "snapshots/megasena/generalistas/transformer_heavy_v1.keras"
model_gen.save(save_path)
print(f"Salvo: {save_path}")
# --- Treinando AutoEncoder Generalista (Fiscal Geral) ---
# Aprende a estrutura 'normal' de QUALQUER sorteio da história.
print("\n--- Treinando AutoEncoder Generalista (Fiscal) ---")
from loterias.models import AutoEncoderModel

model_ae_gen = AutoEncoderModel(1, 60, 6, latent_dim=16)
# Treina com TODOS os dados
model_ae_gen.train(df_full, epochs=50, batch_size=64, verbose=0) 

save_path_ae_gen = "snapshots/megasena/generalistas/autoencoder_fiscal_v1.keras"
model_ae_gen.save(save_path_ae_gen)
print(f"Salvo: {save_path_ae_gen}")
# --- Treinando LSTM Generalista (Deep Learning Clássico) ---
print("\n--- Treinando LSTM Generalista ---")
model_lstm_gen = LSTMModel(1, 60, 6)
model_lstm_gen.train(df_full, epochs=100, batch_size=64)

save_path_lstm_gen = "snapshots/megasena/generalistas/lstm_v1.keras"
model_lstm_gen.save(save_path_lstm_gen)
print(f"Salvo: {save_path_lstm_gen}")

# --- Treinando CatBoost Generalista (Gradient Boosting) ---
print("\n--- Treinando CatBoost Generalista ---")
# Detecta GPU para CatBoost se possível
task_type = "GPU" if len(tf.config.list_physical_devices('GPU')) > 0 else "CPU"
print(f"CatBoost usando: {task_type}")

model_cat_gen = CatBoostModel()
# CatBoostModel.train espera df e params opcionais
# Passamos verbose para acompanhar
model_cat_gen.train(df_full)

save_path_cat_gen = "snapshots/megasena/generalistas/catboost_v1.cbm"
model_cat_gen.save(save_path_cat_gen)
print(f"Salvo: {save_path_cat_gen}")

## 5. Treinar Especialistas (Mega da Virada / Final 0 ou 5)
Filtramos o DataFrame para conter apenas concursos com final 0 ou 5.

In [ ]:
# Filtrar apenas finais 0 ou 5 (ACUMULADOS/Virada)
# Idealmente, filtraríamos pela data (Mês 12, Dia 31) para Mega da Virada específica,
# mas finais 0 ou 5 garantem concursos com prêmios maiores (acumulados).

if 'Concurso' in df_full.columns:
    df_virada = df_full[df_full['Concurso'] % 5 == 0].copy()
else:
    print("Aviso: Coluna 'Concurso' não encontrada. Usando índice.")
    df_virada = df_full[df_full.index % 5 == 0].copy()

print(f"Dados Filtrados (Acumulados - Final 0 ou 5): {len(df_virada)} concursos.")

print("--- Treinando Transformer Especialista (Acumulados) ---")
model_spec = TransformerModel(1, 60, 6)
model_spec.train(df_virada, epochs=200, batch_size=16) 

save_path_spec = "snapshots/megasena/especialistas/virada/transformer_virada_v1.keras"
model_spec.save(save_path_spec)
print(f"Salvo: {save_path_spec}")

# --- Treinando AutoEncoder (Fiscal de Anomalias) para Acumulados ---
# O AutoEncoder aprende o que é um jogo 'normal' dentro desse subgrupo.
# Jogos gerados que fogem muito do padrão (alto reconstruction error) são suspeitos.
print("\n--- Treinando AutoEncoder Especialista (Fiscal) ---")
from loterias.models import AutoEncoderModel

model_ae = AutoEncoderModel(1, 60, 6, latent_dim=16) # Latent dim força compressão
model_ae.train(df_virada, epochs=100, batch_size=32, verbose=0) # Verbose 0 para limpar output

save_path_ae = "snapshots/megasena/especialistas/virada/autoencoder_fiscal_v1.keras"
model_ae.save(save_path_ae)
print(f"Salvo: {save_path_ae}")
# --- Treinando LSTM Especialista (Acumulados) ---
print("\n--- Treinando LSTM Especialista ---")
model_lstm_spec = LSTMModel(1, 60, 6)
model_lstm_spec.train(df_virada, epochs=150, batch_size=32)

save_path_lstm_spec = "snapshots/megasena/especialistas/virada/lstm_virada_v1.keras"
model_lstm_spec.save(save_path_lstm_spec)
print(f"Salvo: {save_path_lstm_spec}")

# --- Treinando CatBoost Especialista (Acumulados) ---
print("\n--- Treinando CatBoost Especialista ---")
model_cat_spec = CatBoostModel()
model_cat_spec.train(df_virada)

save_path_cat_spec = "snapshots/megasena/especialistas/virada/catboost_virada_v1.cbm"
model_cat_spec.save(save_path_cat_spec)
print(f"Salvo: {save_path_cat_spec}")

In [ ]:
# 6. Baixar Snapshots (Google Colab)
# Compacta e baixa a pasta de snapshots automaticamente
import shutil
try:
    from google.colab import files
    shutil.make_archive('snapshots_backup', 'zip', 'snapshots')
    files.download('snapshots_backup.zip')
except ImportError:
    print("Ambiente não é o Colab ou google.colab não encontrado.")